In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

In [2]:
import cvxpy as cp
from measure_optimize import MeasurementOptimizer

(CVXPY) Aug 16 08:08:08 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')
(CVXPY) Aug 16 08:08:08 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')


## Input setting

### Read Q

In [3]:
jaco_info = pd.read_csv('./kinetics_fim/Q.csv')


# remove all initial conditions
Q_stack = []

for i in range(1,9):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(10,18):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(19,27):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
#print(Q_stack)   
print(np.shape(Q_stack))

(24, 4)


### Set Sigma

In [4]:
# variance 
variance = [1, # var_CA^2
            4, # var_CB^2
            8  # var_CC^2
           ]
covariance = [ 0.1, # cov_CA_CB | ti, 
               0.1, # cov_CA_CC | ti, 
               0.5 # cov_CB_CC | ti
]

# construct Sigma
sigma = np.zeros((24, 24))

for i in range(24):
    #for j in range(27):
        
        #if i==j:
    if i<8:
        sigma[i,i]=variance[0]
        sigma[i,i+8]=covariance[0]
        sigma[i,i+16]=covariance[1]
    elif 8<= i < 16:
        sigma[i,i]=variance[1]
        sigma[i,i-8]=covariance[0]
        sigma[i,i+8]=covariance[2]
    elif i>=16:
        sigma[i,i]=variance[2]
        sigma[i,i-16]=covariance[1]
        sigma[i,i-8]=covariance[2]
        
print(np.shape(sigma))

(24, 24)


### Other settings

In [5]:
number_measure = 3 
number_t = 8
cost = [10]*number_t + [6]*number_t*2 

## Check square problem solutions

In [6]:
ya1 = [1, 1, 1, 1, 0, 0, 0, 0]
yb1 = [1, 1, 1, 0.8895839685893835, 0, 1, 1, 1]
yc1 = [1, 0.5873479124974504, 0, 0, 0, 0, 0.5230594984957986, 1]


solution1 = ya1 + yb1 + yc1 
print(np.shape(solution1))

(24,)


In [7]:
calculator = MeasurementOptimizer(Q_stack, number_measure, number_t, cost, error_cov = sigma)

calculator.compute_FIM(solution1)

Q shape: 24 4
FIM unit for the two measurements:  [[ 1.81989369e+00  2.25049538e-13 -3.00462517e+00  2.25049538e-13]
 [ 2.25049538e-13  2.78298094e-26 -3.71554400e-13  2.78298094e-26]
 [-3.00462517e+00 -3.71554400e-13  4.96060429e+00 -3.71554400e-13]
 [ 2.25049538e-13  2.78298094e-26 -3.71554400e-13  2.78298094e-26]]
FIM unit for the two measurements:  [[-3.63857449e-17 -7.77377216e-30  7.01780234e-17 -5.83032912e-30]
 [-4.49949090e-30 -9.61311006e-43  8.67827162e-30 -7.20983255e-43]
 [ 6.00724789e-17  1.28344154e-29 -1.15863172e-16  9.62581152e-30]
 [-4.49949090e-30 -9.61311006e-43  8.67827162e-30 -7.20983255e-43]]
FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0.

FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
FIM unit for the two measurements:  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
FIM unit for the two measurements:  [[ 7.29344965e-18 -5.03620857e-16 -1.71452163e-17  1.60389617e-15]
 [ 1.55823705e-30 -1.07598011e-28 -3.66305557e-30  3.42670554e-28]
 [-1.40670441e-17  9.71345134e-16  3.30683732e-17 -3.09347144e-15]
 [ 1.16867779e-30 -8.06985085e-29 -2.74729168e-30  2.57002916e-28]]
FIM unit for the two measurements:  [[-6.72129564e-03 -3.41796135e-02  4.12682824e-03  1.19887266e-01]
 [-1.43599701e-15 -7.30243474e-15  8.81692062e-16  2.56137752e-14]
 [ 1.29635175e-02  6.59230069e-02 -7.95950857e-03 -2.31229328e-01]
 [-1.07699776e-15 -5.47682605e-15  6.61269046e-16  1.92103314e-14]]
FI

array([[  4.98294842,   1.86384141,  -8.89907613,  -7.76576957],
       [  1.86384141,   4.24425089,  -2.72593399, -17.02417346],
       [ -8.89907613,  -2.72593399,  16.21336352,  11.49758729],
       [ -7.76576957, -17.02417346,  11.49758729,  69.63788722]])

## Optimize